# day 17

In [ ]:
from copy import deepcopy

## part 1

In [ ]:
g = """.#.
..#
###"""

In [ ]:
def parse(grid):
    x, y = 0, 0
    cells = set()
    for line in grid.split("\n"):
        y = 0
        for char in line:
            if char == '#':
                cells.add((x, y, 0))
            y += 1
        x += 1
    return cells

In [ ]:
G = parse(g)
G

{(0, 1, 0), (1, 2, 0), (2, 0, 0), (2, 1, 0), (2, 2, 0)}

Get grid edges:

In [ ]:
def dim_min(G, d): return min(G, key=lambda x: x[d])[d]
def dim_max(G, d): return max(G, key=lambda x: x[d])[d]
def dim_iter(G, d):
    for i in range(dim_min(G, d) - 1, dim_max(G, d) + 2):
        yield i

Get 26 neighbours of each cell:

In [ ]:
def neighbours(x, y, z):
    return [
        (_x, _y, _z)
        for _x in [x - 1, x, x + 1]
        for _y in [y - 1, y, y + 1]
        for _z in [z - 1, z, z + 1]
        if (_x, _y, _z) != (x, y, z)
    ]

In [ ]:
def active_neighbours(G, x, y, z):
    c = 0
    for n in neighbours(x, y, z):
        if n in G:
            c += 1
    return c

And check if we can iterate over them:

In [ ]:
for x in dim_iter(G, 0):
    for y in dim_iter(G, 1):
        for z in dim_iter(G, 2):
            print(f"{x:2d}, {y:2d}, {z:2d}", ': ', '#' if (x, y, z) in G else '.', active_neighbours(G, x, y, z))

-1, -1, -1 :  . 0
-1, -1,  0 :  . 0
-1, -1,  1 :  . 0
-1,  0, -1 :  . 1
-1,  0,  0 :  . 1
-1,  0,  1 :  . 1
-1,  1, -1 :  . 1
-1,  1,  0 :  . 1
-1,  1,  1 :  . 1
-1,  2, -1 :  . 1
-1,  2,  0 :  . 1
-1,  2,  1 :  . 1
-1,  3, -1 :  . 0
-1,  3,  0 :  . 0
-1,  3,  1 :  . 0
 0, -1, -1 :  . 0
 0, -1,  0 :  . 0
 0, -1,  1 :  . 0
 0,  0, -1 :  . 1
 0,  0,  0 :  . 1
 0,  0,  1 :  . 1
 0,  1, -1 :  . 2
 0,  1,  0 :  # 1
 0,  1,  1 :  . 2
 0,  2, -1 :  . 2
 0,  2,  0 :  . 2
 0,  2,  1 :  . 2
 0,  3, -1 :  . 1
 0,  3,  0 :  . 1
 0,  3,  1 :  . 1
 1, -1, -1 :  . 1
 1, -1,  0 :  . 1
 1, -1,  1 :  . 1
 1,  0, -1 :  . 3
 1,  0,  0 :  . 3
 1,  0,  1 :  . 3
 1,  1, -1 :  . 5
 1,  1,  0 :  . 5
 1,  1,  1 :  . 5
 1,  2, -1 :  . 4
 1,  2,  0 :  # 3
 1,  2,  1 :  . 4
 1,  3, -1 :  . 2
 1,  3,  0 :  . 2
 1,  3,  1 :  . 2
 2, -1, -1 :  . 1
 2, -1,  0 :  . 1
 2, -1,  1 :  . 1
 2,  0, -1 :  . 2
 2,  0,  0 :  # 1
 2,  0,  1 :  . 2
 2,  1, -1 :  . 4
 2,  1,  0 :  # 3
 2,  1,  1 :  . 4
 2,  2, -1 :  . 3
 2,  2,  0

Now apply the rules:
- If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
- If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.

In [ ]:
def evolve(G, num_cycles):
    G2 = deepcopy(G)
    for _ in range(num_cycles):
        for x in dim_iter(G2, 0):
            for y in dim_iter(G2, 1):
                for z in dim_iter(G2, 2):
                    a = active_neighbours(G2, x, y, z)
                    if (x, y, z) in G2:
                        if a not in (2, 3):
                            G2.discard((x, y, z))
                    else:
                        if a == 3:
                            G2.add((x, y, z))
    return G2

In [ ]:
len(evolve(G, 1))

19

This is incorrect -- it looks like the problem lies in not applying the changes **at the same time**.  Elements are added to and removed from the grid during the loop, while they should be counted first, with rules applied later.

Second attempt:

In [ ]:
def evolve(G, n):
    G2 = deepcopy(G)
    for _ in range(n):
        ns = {}
        for x in dim_iter(G2, 0):
            for y in dim_iter(G2, 1):
                for z in dim_iter(G2, 2):
                    ns[(x, y, z)] = active_neighbours(G2, x, y, z)

        for ((x, y, z), a) in ns.items():
            if (x, y, z) in G2:
                if a not in (2, 3):
                    G2.discard((x, y, z))
            else:
                if a == 3:
                    pass
                    G2.add((x, y, z))
    return G2

In [ ]:
len(evolve(G, 1))

11

As expected.

In [ ]:
assert len(evolve(G, 1)) == 11

In [ ]:
def p1(G): return len(evolve(G, 6))

In [ ]:
assert p1(G) == 112

In [ ]:
g = """#......#
##.#..#.
#.#.###.
.##.....
.##.#...
##.#....
#####.#.
##.#.###"""
G = parse(g)
G

{(0, 0, 0),
 (0, 7, 0),
 (1, 0, 0),
 (1, 1, 0),
 (1, 3, 0),
 (1, 6, 0),
 (2, 0, 0),
 (2, 2, 0),
 (2, 4, 0),
 (2, 5, 0),
 (2, 6, 0),
 (3, 1, 0),
 (3, 2, 0),
 (4, 1, 0),
 (4, 2, 0),
 (4, 4, 0),
 (5, 0, 0),
 (5, 1, 0),
 (5, 3, 0),
 (6, 0, 0),
 (6, 1, 0),
 (6, 2, 0),
 (6, 3, 0),
 (6, 4, 0),
 (6, 6, 0),
 (7, 0, 0),
 (7, 1, 0),
 (7, 3, 0),
 (7, 5, 0),
 (7, 6, 0),
 (7, 7, 0)}

In [ ]:
p1(G)

271

## part 2
Just add one more dimension:

In [ ]:
def parse2(grid):
    x, y = 0, 0
    cells = set()
    for line in grid.split("\n"):
        y = 0
        for char in line:
            if char == '#':
                cells.add((x, y, 0, 0))
            y += 1
        x += 1
    return cells

In [ ]:
def neighbours2(x, y, z, w):
    return [
        (_x, _y, _z, _w)
        for _x in [x - 1, x, x + 1]
        for _y in [y - 1, y, y + 1]
        for _z in [z - 1, z, z + 1]
        for _w in [w - 1, w, w + 1]
        if (_x, _y, _z, _w) != (x, y, z, w)
    ]

In [ ]:
def active_neighbours2(G, x, y, z, w):
    c = 0
    for n in neighbours2(x, y, z, w):
        if n in G:
            c += 1
    return c

In [ ]:
def evolve2(G, n):
    G2 = deepcopy(G)
    for _ in range(n):
        ns = {}
        for x in dim_iter(G2, 0):
            for y in dim_iter(G2, 1):
                for z in dim_iter(G2, 2):
                    for w in dim_iter(G2, 3):
                        ns[(x, y, z, w)] = active_neighbours2(G2, x, y, z, w)

        for ((x, y, z, w), a) in ns.items():
            if (x, y, z, w) in G2:
                if a not in (2, 3):
                    G2.discard((x, y, z, w))
            else:
                if a == 3:
                    pass
                    G2.add((x, y, z, w))
    return G2

In [ ]:
def p2(G): return len(evolve2(G, 6))

In [ ]:
g = """.#.
..#
###"""
G = parse2(g)

In [ ]:
assert len(evolve2(G, 1)) == 29

In [ ]:
g = """#......#
##.#..#.
#.#.###.
.##.....
.##.#...
##.#....
#####.#.
##.#.###"""
G = parse2(g)

In [ ]:
p2(G)

2064